In [70]:
import sys
from pathlib import Path
#from mlfs import config

#Find root directory
root_dir = Path().absolute()
if root_dir.parts[-2:] == ('notebooks', 'algae_bloom'):
    root_dir = Path(*root_dir.parts[:-2])
root_dir = str(root_dir)

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")


Root dir: /Users/kevinkokalari/Documents/Skalbar Maskininlärning och Djupinlärning/Laborationer/id2223-project


<span style="font-width:bold; font-size: 3rem; color:#333;">- Daily Feature Pipeline for Water Temperature and Weather</span>

## 🌐 Imports

In [71]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## 🙊 Get variebles from Hopsworks secret </span>


In [72]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

#Get the dictionary with all batch locations
bath_locations_json = secrets.get_secret("BATH_LOCATIONS_JSON").value
bath_locations = json.loads(bath_locations_json)
print(bath_locations)

#Set date to today
today = datetime.date.today()

2025-12-19 10:30:50,525 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-12-19 10:30:50,532 INFO: Initializing external client
2025-12-19 10:30:50,532 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-12-19 10:30:55,935 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286295
[{'bath_location': 'Bergabadet', 'latitude': 59.057008, 'longitude': 17.440774}, {'bath_location': 'Bränningestrand', 'latitude': 59.148617, 'longitude': 17.6674}, {'bath_location': 'Eklundsnäsbadet', 'latitude': 59.16883, 'longitude': 17.59184}, {'bath_location': 'Farstanäsbadet', 'latitude': 59.096884, 'longitude': 17.65387}, {'bath_location': 'Mälarbadet', 'latitude': 59.222657, 'longitude': 17.611886}, {'bath_location': 'Nya Malmsjöbadet', 'latitude': 59.234823, 'longitude': 17.536534}, {'bath_location': 'Näsets udde(Glashyttan)', 'latitude': 59.158419, 'longitude': 17.66072}, {'bath_location': 'Underåsbadet', 'latitude': 59.26482, 'longitude': 17.536534}, {'bath_location': 'Åbynäsbadet', 'latitude': 59.018397, 'longitude': 17.619576}]


## ☂️ Get feature groups

In [73]:
#Select version of feature group
wt_version = 1
w_version = 1

#Retrieve feature groups
water_temperature_fg = fs.get_feature_group( 
    name='water_temperature',
    version=wt_version,
)

weather_fg = fs.get_feature_group(
    name='weather',
    version=w_version,
)

---

## 💦 Get today's water temperature data


In [74]:
import requests
import pandas as pd

wt_today_df = util.get_wt()

wt_today_df

,temp_water,formatted_time,alias,latitude,longitude


In [75]:
wt_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   temp_water      0 non-null      float64       
 1   formatted_time  0 non-null      datetime64[ns]
 2   alias           0 non-null      object        
 3   latitude        0 non-null      float64       
 4   longitude       0 non-null      float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 132.0+ bytes


## 🔙 Add lagged water temperature

In [76]:
#Load dataframe and covert dates to not include time
features_df = water_temperature_fg.read()
features_df["formatted_time"] = pd.to_datetime(features_df["formatted_time"])#.dt.date
features_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.46s) 


,formatted_time,alias,temp_water,longitude,latitude,lagged_wt_1_day,lagged_wt_2_days,lagged_wt_3_days
0,2025-02-27 13:53:00+00:00,Åbynäsbadet,2.0,17.619576,59.018397,1.9,1.7,1.7
1,2022-10-26 10:40:00+00:00,Eklundsnäsbadet,8.3,17.591840,59.168830,11.3,8.4,5.3
2,2022-12-26 14:33:00+00:00,Nya Malmsjöbadet,1.9,17.536534,59.234823,1.9,1.9,1.9
3,2022-07-22 11:44:00+00:00,Bränningestrand,20.8,17.667400,59.148617,20.4,18.9,18.8
4,2022-12-11 10:11:00+00:00,Farstanäsbadet,3.9,17.653870,59.096884,5.0,4.4,4.3
...,...,...,...,...,...,...,...,...
6961,2023-03-30 10:32:00+00:00,Näsets udde(Glashyttan),1.6,17.660720,59.158419,1.2,1.9,2.2
6962,2025-01-23 13:50:00+00:00,Åbynäsbadet,1.6,17.619576,59.018397,1.5,1.5,1.5
6963,2022-02-26 12:08:00+00:00,Bränningestrand,1.4,17.667400,59.148617,1.2,1.3,0.9
6964,2023-02-27 15:40:00+00:00,Åbynäsbadet,1.5,17.619576,59.018397,1.5,1.8,2.0


In [77]:
yesterday = today - datetime.timedelta(days=1)
two_days_ago = today - datetime.timedelta(days=2)
three_days_ago = today - datetime.timedelta(days=3)
print("yesterday", yesterday)
print("two_days_ago", two_days_ago)
print("three_days_ago", three_days_ago)

yesterday 2025-12-18
two_days_ago 2025-12-17
three_days_ago 2025-12-16


In [78]:
wt_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   temp_water      0 non-null      float64       
 1   formatted_time  0 non-null      datetime64[ns]
 2   alias           0 non-null      object        
 3   latitude        0 non-null      float64       
 4   longitude       0 non-null      float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 132.0+ bytes


In [79]:


# One row per (street, date) for each lag
yesterday_df = (features_df[features_df["formatted_time"].dt.date == yesterday][["alias", "temp_water"]].rename(columns={"temp_water": "lagged_wt_1_day"}))

two_days_ago_df = (features_df[features_df["formatted_time"].dt.date == two_days_ago][["alias", "temp_water"]].rename(columns={"temp_water": "lagged_wt_2_days"}))

three_days_ago_df = (features_df[features_df["formatted_time"].dt.date == three_days_ago][["alias", "temp_water"]].rename(columns={"temp_water": "lagged_wt_3_days"}))

# Merge lagged values into today's rows, per street/sensor
wt_today_df = (wt_today_df.merge(yesterday_df, on="alias", how="left").merge(two_days_ago_df, on="alias", how="left").merge(three_days_ago_df, on="alias", how="left"))


wt_today_df

,temp_water,formatted_time,alias,latitude,longitude,lagged_wt_1_day,lagged_wt_2_days,lagged_wt_3_days


## 🌦 Get weather data

In [ ]:
daily_df = None


for index, row in wt_today_df.iterrows():

    current_datetime = row["formatted_time"]
    rounded_datetime = row["rounded"].dt.round("H")

    lower_bound = current_datetime - pd.Timedelta(minutes=1)
    upper_bound = current_datetime + pd.Timedelta(minutes=1)

    hourly_df = util.get_hourly_weather_forecast(row["latitude"], row["longitude"]).rename({"date":"rounded"}, inplace=True)
    hourly_df = hourly_df.set_index('rounded')

    # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
    daily_df_partial = hourly_df.between_time(lower_bound, upper_bound)
    daily_df_partial = daily_df_partial.reset_index()

    #daily_df_partial['formatted_time'] = pd.to_datetime(daily_df_partial['formatted_time']).dt.date
    daily_df_partial['rounded'] = pd.to_datetime(daily_df_partial['rounded'])
    daily_df_partial['formatted_time'] = current_datetime

    if daily_df is None:
        daily_df = daily_df_partial
    else:
        daily_df.concat([daily_df, daily_df_partial], ignore_index=True)
        

print(daily_df)

None


In [83]:
if not daily_df is None:
    daily_df.info()

## 🆙 Upload data to feature groups

In [ ]:
# Insert new data
water_temperature_fg.insert(wt_today_df)

2025-11-18 18:31:17,915 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1703474


Uploading Dataframe: 100.00% |██████████| Rows 6/6 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/air_quality_3_offline_fg_materialization/executions


(Job('air_quality_3_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747713
         }
       },
       "result": {
         "observed_value": 6.0,
         "element_count": 6,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T05:31:17.000914Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [ ]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 18:31:32,448 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1703478


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/weather_2_offline_fg_materialization/executions
2025-11-18 18:31:48,883 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 18:31:52,176 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 18:33:26,875 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-18 18:33:30,228 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-18 18:33:30,777 INFO: Waiting for log aggregation to finish.
2025-11-18 18:33:30,778 INFO: Execution finished successfully.


(Job('weather_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747717
         }
       },
       "result": {
         "observed_value": 5.351784706115723,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T05:31:32.000447Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 